In [18]:
import pandas as pd
import os
import dotenv

dotenv.load_dotenv()
BASE_DIR = os.getcwd()

# Import Excel

In [19]:
excel_path = "disposition_code_ref_table.xlsx"

df = pd.read_excel(os.path.join(BASE_DIR, excel_path))
df.drop(columns=["Konkatenasi_Layer123"], inplace=True)
df

,Layer 1,Layer 2,Layer 3
0,Complain (General Customer),Branchless,BLB: ATM Barang Jatuh Di Area ATM
1,Complain (General Customer),Branchless,BLB: ATM Gagal Tarik
2,Complain (General Customer),Branchless,BLB: ATM Gagal Transaksi
3,Complain (General Customer),Branchless,BLB: ATM Jumlah Uang Tidak Sesuai
4,Complain (General Customer),Branchless,BLB: ATM Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,KTA: Sales Done CC KTP Non Leads
1164,Sales,KTA,KTA: Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/SPV


## Get data By Product Name

In [20]:
df["Layer 3"].value_counts()

Layer 3
Others                             7
CC: Wrong Number                   7
CC: Tidak Setuju                   7
CC: Setuju                         7
CC: Other Respond                  7
                                  ..
FND: Deposito Status               1
FND: Tabungan Kendala Transaksi    1
FND: Giro Status                   1
FND: Tabungan Status               1
Kontak Other Places                1
Name: count, Length: 584, dtype: int64

In [21]:
get_data_by_product = df.loc[
    (df["Layer 2"] == "Credit Card")
    & (df["Layer 1"].str.contains("Complain"))
]

distinct_values = len(df["Layer 3"].unique())

## Create JSON from action to product

In [22]:
import json

action_to_product = {}

for _, row in df.iterrows():
    action, product, detail = row.iloc[0].strip(), row.iloc[1].strip(), row.iloc[2].strip()

    if action not in action_to_product:
        action_to_product[action] = []

    if product not in action_to_product[action]:
        action_to_product[action].append(product)

action_to_product_json = json.dumps(action_to_product, indent=2)

# create json file
with open("action_to_product.json", "w") as f:
    f.write(action_to_product_json)

## Create JSON from product to action

In [23]:
import json

product_to_action = {}

for _, row in df.iterrows():
    action, product, detail = row.iloc[0].strip(), row.iloc[1].strip(), row.iloc[2].strip()

    if product not in product_to_action:
        product_to_action[product] = []

    if action not in product_to_action[product]:
        product_to_action[product].append(action)

product_to_action_json = json.dumps(product_to_action, indent=2)

# create json file
with open("product_to_action.json", "w") as f:
    f.write(product_to_action_json)

## Create JSON from (action_product) to detail

In [24]:
action_product_to_detail = {}

for _, row in df.iterrows():
    action_product, detail =f"{row.iloc[0].strip()}|{row.iloc[1].strip()}", row.iloc[2].strip()

    if action_product not in action_product_to_detail:
        action_product_to_detail[action_product] = []

    if detail not in action_product_to_detail[action_product]:
        action_product_to_detail[action_product].append(detail)

result = json.dumps(action_product_to_detail, indent=2)

# create json file
with open("action_product_to_detail.json", "w") as f:
    f.write(result)

# Create Excell from Ref Table Cleaned

In [25]:
df["Konkatenasi_Layer123"] = df["Layer 1"] + "|" + df["Layer 2"] + "|" + df["Layer 3"]

df.to_excel("disposition_code_ref_table_cleaned.xlsx", index=False)

# READ JSON FILE

In [26]:
# load json file
with open('action_to_product.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data.items(), columns=["Action", "Product"])

In [27]:
df["Product_Count"] = df["Product"].apply(len)
df

,Action,Product,Product_Count
0,Complain (General Customer),"[Branchless, Credit Card, Funding, Lending, Ot...",10
1,Compliment (General Customer),"[Branchless, Credit Cards, Funding, Lending, O...",6
2,Feedback/Report (General Customer),"[Branchless, Credit Cards, Funding, Lending, O...",6
3,Inquiry (General Customer),"[Branchless, Credit Card, Funding, Lending, Ot...",10
4,Request (General Customer),"[Branchless, Credit Cards, Funding, Lending, O...",10
5,Spam (General Customer),[Spam],1
6,Report Status Checking,"[Branchless, Credit Cards, Funding, Lending, O...",6
7,Campaign,"[CNAF, Credit Cards General, Telesales Dok KTP...",19
8,Ongoing,"[Chat lanjutan, Info belum lengkap, Salah laya...",3
9,Miscellaneous,[Salah Nama],1


# READ JSON

In [31]:
# load json file
with open('action_product_to_detail.json', 'r') as f:
    data = json.load(f)

max_option_layer_3 = len(max(data.values(), key=len))
max_option_layer_3

69

In [30]:
with open('action_to_product.json', 'r') as f:
    data = json.load(f)

max_option_layer_2 = len(max(data.values(), key=len))
max_option_layer_2

19